This notebook takes in a list of galaxies and the voids found using $V^2$ and VoidFinder and appends to the list of galaxies columns that denote which are void, wall, edge, and outside the survey for each algorithm.

In [1]:
from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import astropy.constants as const

from vast.voidfinder.distance import z_to_comoving_dist
from vast.voidfinder.vflag import determine_vflag

import numpy as np

import pickle

import matplotlib.pyplot as plt
%matplotlib notebook

#### Miscellaneous functions

In [2]:
def flatten(array): # when np.flatten() doesn't work
    temp = []
    for sublist in array:
        for item in sublist:
            temp.append(item)
    return np.array(temp)

In [3]:
def rmv(array,out): # removing galaxies in "out" from galaxies in "array"
    '''
    test_out =[] 
    for i in array:
        if i in out:
            test_out.append(False)
        else:
            test_out.append(True)
    '''
    test_out = np.ones(len(array), dtype=bool)
    
    for i,obj in enumerate(array):
        if obj in out:
            test_out[i] = False
    
    return np.array(array)[test_out]

# Galaxies

In [4]:
#gal_filename = 'data_all.txt'
#data = Table.read('data_all.txt', format='ascii')

gal_filename = '../../../../data/nsa_v1_0_1_VAGC.fits'

hdu = fits.open(gal_filename)
data = Table(hdu[1].data)
hdu.close()

In [ ]:
data[:5]

# $V^2$ classification

In [5]:
#galzones_filename = 'DR7_galzones.dat'
#zonevoids_filename = 'DR7_zonevoids.dat'

V2_gal_filename = '../../../void_catalogs/public/v1.0.0/nsa_v1_0_1_main.txt'
galzones_filename = '../../../void_catalogs/public/v1.0.0/V2_VIDE-nsa_v1_0_1_galzones.dat'
zonevoids_filename = '../../../void_catalogs/public/v1.0.0/V2_VIDE-nsa_v1_0_1_zonevoids.dat'

V2_gals = Table.read(V2_gal_filename, format='ascii.commented_header')
#galzones_old = Table.read('old_data/old/DR7_galzones.dat', format = 'ascii')
galzones = Table.read(galzones_filename, format='ascii.commented_header') 
zonevoids = Table.read(zonevoids_filename, format='ascii.commented_header')

total_glx = len(galzones)

In [6]:
#gal_indx = galzones_old['gal']
gal_indx = galzones['gal']

In [ ]:
print(total_glx)

In [ ]:
print(len(V2_gals))

So we need to determine which galaxies from the input galaxy catalog remain in the volume-limited catalog to figure out the $V^2$ classification.  Ugh...

##### Identify objects in the volume-limited catalog, and add their index number to a column in `galzones`

In [7]:
vollim_boolean = (V2_gals['redshift'] <= 0.114) & (V2_gals['rabsmag'] <= -20.0)

vollim_V2_gals = V2_gals[vollim_boolean]

print(len(vollim_V2_gals))

194125


In [8]:
data_coords = SkyCoord(ra=data['RA']*u.degree, 
                       dec=data['DEC']*u.degree, 
                       radial_velocity=const.c*data['Z'])
vollim_V2_coords = SkyCoord(ra=vollim_V2_gals['ra']*u.degree, 
                            dec=vollim_V2_gals['dec']*u.degree, 
                            radial_velocity=const.c*vollim_V2_gals['redshift'])

idx, d2d, d3d = vollim_V2_coords.match_to_catalog_sky(data_coords)

galzones['i_NSA'] = idx

#### Edge and outside galaxies

In [9]:
edge_bool = [bool(x) for x in np.array(np.array(galzones['edge']))] 
edge_gal = np.array(gal_indx[edge_bool])

In [10]:
out_bool = [bool(x) for x in np.array(np.array(galzones['out']))]
out_gal = np.array(gal_indx[out_bool])

In [11]:
len(out_gal)+len(edge_gal)

12981

#### Void galaxies

In [12]:
void_zones = zonevoids[(zonevoids['void0'] != -1) & (zonevoids['void1'] != -1)]['zone'] # zones that are in voids

In [13]:
void_gal_ = []
for i in void_zones:
    void_gal_.append(list(gal_indx[galzones['zone'] == i]))

void_gal_flat = flatten(void_gal_) # around 200,000 are void galaxies <-- updated to 125,062

In [14]:
#void_gal_flat[199377] # galaxy that's outside that is in zone that is in void

#### Fact #1: edge and outside galaxies exist in zones that are in voids

In [15]:
void_gal = rmv(void_gal_flat,edge_gal) # eliminating edge galaxies, True for non edge, about 8000

In [16]:
void_gal = rmv(void_gal,out_gal)

In [17]:
np.intersect1d(void_gal,out_gal , return_indices=True) 

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

#### wall galaxies

In [18]:
non_void_zones = zonevoids[(zonevoids['void0'] == -1)]['zone'] # zones that are not in voids

non_void_gal = []
for i in non_void_zones:
    non_void_gal.append(list(gal_indx[galzones['zone'] == i]))
        
wall_gal = rmv(flatten(non_void_gal),edge_gal)

In [19]:
np.intersect1d(wall_gal,edge_gal , return_indices=True) 

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [20]:
wall_gal = rmv(wall_gal,out_gal)

### Check

In [21]:
len(wall_gal)+len(void_gal)+len(edge_gal)+len(out_gal), total_glx

(194125, 194125)

In [22]:
wall_gal

array([  6131,   6140,   6153, ...,  24804, 174257,  69783])

### Add vflag column to main data array

For objects that are not in the main footprint, they are classified as outside.

In [23]:
data['vflag_V2'] = 9 # Initialize the vflag column for V2, setting the default to outside the survey mask

data['vflag_V2'][galzones['i_NSA'][wall_gal]] = 0
data['vflag_V2'][galzones['i_NSA'][void_gal]] = 1
data['vflag_V2'][galzones['i_NSA'][edge_gal]] = 2
data['vflag_V2'][galzones['i_NSA'][out_gal]] = 9

For objects that are in the main footprint but not in the volume-limited sample, they get the classification of their nearest neighbor that is in the volume-limited catalog.

In [24]:
V2_coords = SkyCoord(ra=V2_gals['ra']*u.degree, 
                     dec=V2_gals['dec']*u.degree, 
                     radial_velocity=const.c*V2_gals['redshift'])

idx_vollim,_,_ = V2_coords.match_to_catalog_sky(vollim_V2_coords)

idx_data,_,_ = V2_coords.match_to_catalog_sky(data_coords)

data['vflag_V2'][idx_data] = data['vflag_V2'][idx[idx_vollim]]

# VoidFinder classification

In [25]:
'''
data_voidfinder = Table.read('ALL_vflag_comoving.txt', format='ascii')

wall_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 0)
void_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 1)

out_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 9)
edge_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 2)
''';

In [26]:
VF_holes_filename = '../../../void_catalogs/public/v1.0.0/VoidFinder-nsa_v1_0_1_main_comoving_holes.txt'

VF_holes = Table.read(VF_holes_filename, format='ascii.commented_header')

In [27]:
mask_filename = '../../../void_catalogs/SDSS/VoidFinder/python_implementation/NSA_main_mask.pickle'

mask_infile = open(mask_filename, 'rb')
mask, mask_resolution = pickle.load(mask_infile)
mask_infile.close()

##### Convert galaxy (ra, dec, z) to (x, y, z)

In [28]:
DtoR = np.pi/180.

s = z_to_comoving_dist(data['Z'].data.astype(np.float32), 0.315, 1)

x = s*np.cos(data['DEC']*DtoR)*np.cos(data['RA']*DtoR)
y = s*np.cos(data['DEC']*DtoR)*np.sin(data['RA']*DtoR)
z = s*np.sin(data['DEC']*DtoR)

### Add vflag column to main data array

In [29]:
data['vflag_VF'] = 9

for i in range(len(data)):
    data['vflag_VF'][i] = determine_vflag(x[i], y[i], z[i], VF_holes, mask, mask_resolution)

In [30]:
data[:5]

IAUNAME,SUBDIR,RA,DEC,ISDSS,INED,ISIXDF,IALFALFA,IZCAT,ITWODF,MAG,Z,ZSRC,SIZE,RUN,CAMCOL,FIELD,RERUN,XPOS,YPOS,NSAID,ZDIST,SERSIC_NMGY [7],SERSIC_NMGY_IVAR [7],SERSIC_OK,SERSIC_RNMGY [7],SERSIC_ABSMAG [7],SERSIC_AMIVAR [7],EXTINCTION [7],SERSIC_KCORRECT [7],SERSIC_KCOEFF [5],SERSIC_MTOL [7],SERSIC_B300,SERSIC_B1000,SERSIC_METS,SERSIC_MASS,XCEN,YCEN,NPROF [7],"PROFMEAN [15,7]","PROFMEAN_IVAR [15,7]","QSTOKES [15,7]","USTOKES [15,7]","BASTOKES [15,7]","PHISTOKES [15,7]",PETRO_FLUX [7],PETRO_FLUX_IVAR [7],FIBER_FLUX [7],FIBER_FLUX_IVAR [7],PETRO_BA50,PETRO_PHI50,PETRO_BA90,PETRO_PHI90,SERSIC_FLUX [7],SERSIC_FLUX_IVAR [7],SERSIC_N,SERSIC_BA,SERSIC_PHI,ASYMMETRY [7],CLUMPY [7],DFLAGS [7],AID,PID,DVERSION,PROFTHETA [15],PETRO_THETA,PETRO_TH50,PETRO_TH90,SERSIC_TH50,PLATE,FIBERID,MJD,RACAT,DECCAT,ZSDSSLINE,SURVEY,PROGRAMNAME,PLATEQUALITY,TILE,PLUG_RA,PLUG_DEC,ELPETRO_BA,ELPETRO_PHI,ELPETRO_FLUX_R,ELPETRO_FLUX_IVAR_R,ELPETRO_THETA_R,ELPETRO_TH50_R,ELPETRO_TH90_R,ELPETRO_THETA,ELPETRO_FLUX [7],ELPETRO_FLUX_IVAR [7],ELPETRO_TH50 [7],ELPETRO_TH90 [7],ELPETRO_APCORR_R,ELPETRO_APCORR [7],ELPETRO_APCORR_SELF [7],ELPETRO_NMGY [7],ELPETRO_NMGY_IVAR [7],ELPETRO_OK,ELPETRO_RNMGY [7],ELPETRO_ABSMAG [7],ELPETRO_AMIVAR [7],ELPETRO_KCORRECT [7],ELPETRO_KCOEFF [5],ELPETRO_MASS,ELPETRO_MTOL [7],ELPETRO_B300,ELPETRO_B1000,ELPETRO_METS,IN_DR7_LSS,u_r,g_r,NUV_r,index,imc,aimc,cd,conx1,u_r_KIAS,prmag,BPTclass,SFR,sSFR,HImass,flag3727,t3,Z12logOH,Zerr,N12logNH,NHerr,logNO,NOerr,Ne12logNeH,NeHerr,logNeO,NeOerr,Z_N2O2,vorvol,vflag_V2,vflag_VF
str19,str27,float64,float64,int32,int32,int32,int32,int32,int32,float32,float32,str7,float32,int16,uint8,int16,str3,float32,float32,int32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,str8,float32,float32,float32,float32,float32,int32,int16,int32,float64,float64,float32,str6,str27,str8,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64
J094651.40-010228.5,09h/m00/J094651.40-010228.5,146.714215072841,-1.0412800233313741,0,72212,37354,-1,-1,-1,15.178774,0.021222278,sdss,0.07,756,1,206,301,136.2967,1095.152,0,0.020687785,29.696056 .. 3267.6895,0.29814342 .. 0.00012172792,1,31.368013 .. 3501.2527,-15.17281 .. -20.302134,223.03555 .. 1102.6129,0.4536473 .. 0.080934174,-0.005893635 .. 0.019576037,0.00017777947 .. 5.1533486e-11,0.00020792194 .. 0.98780394,2.7473723e-05,0.31195974,0.035135116,8898397000.0,215.39407348632812,213.4535369873047,10 .. 12,0.3149959 .. 0.0,233.37904 .. 0.0,0.062215745 .. -0.07950058,0.0006146176 .. -0.1274703,0.8828513 .. 0.7387829,0.2829979 .. -60.97547,18.203371 .. 2264.6604,1.9946122 .. 0.015697604,1.0260131 .. 561.97687,47.3397 .. 0.33574256,0.88909996,14.777527,0.80408496,17.367554,19.554192 .. 3146.785,1.0691423 .. 0.017162137,4.7761517,0.6651653,15.97821,-0.0109440535 .. 0.004525926,0.042470127 .. 0.042621203,0 .. 0,0,36,v2_1_13,0.22341923 .. 258.39,7.2478933,3.4641922,10.453795,5.882104,266,1,51630,146.71420341874853,-1.0412749124036818,0.0,sdss,legacy,good,122,146.71421,-1.0413043,0.80408496,17.367554,1144.0713,0.4582725,7.3913364,3.7061903,10.666219,7.3913364,18.787916 .. 2225.009,1.8480047 .. 0.017611798,5.8557696 .. 3.4004514,11.650886 .. 10.1460905,0.9986329,1.0687045 .. 1.001099,1.0695114 .. 1.0010818,28.532349 .. 2310.4973,0.42499655 .. 0.00024172392,1,28.661283 .. 2337.7668,-15.124495

### Check

In [ ]:
#len(out_gal_vf)+len(edge_gal_vf)+len(wall_gal_vf)+len(void_gal_vf) == len(data_voidfinder)

### add flag columns and save file

In [ ]:
'''
flag_vsquared=[] # checks all galaxies in each statement as a double check for any overlaps
for i in range(len(data)):
    if i in wall_gal:
        flag_vsquared.append(0)
    if i in void_gal:
        flag_vsquared.append(1)
    if i in edge_gal:
        flag_vsquared.append(2)
    if i in out_gal:
        flag_vsquared.append(9)
    
# flag_vsquared_vf=[]
# for i in range(len(data)):
#     if i in wall_gal_vf:
#         flag_vsquared_vf.append(0)
#     if i in void_gal_vf:
#         flag_vsquared_vf.append(1)
#     if i in edge_gal_vf:
#         flag_vsquared_vf.append(2)
#     if i in out_gal_vf:
#         flag_vsquared_vf.append(9)
        
print(len(flag_vsquared),len(flag_vsquared_vf), len(data)) #check
'''

In [32]:
'''
flags_table = Table([flag_vsquared_vf,flag_vsquared], names=('flag_voidfinder', 'flag_vsquared'))

data.add_columns(flags_table.columns.values())

ascii.write(data, 'data_flags_updated_Nov1.dat')
'''
data.write('../../../../data/NSA_v1_0_1_VAGC_vflag-V2-VF.fits', format='fits')